In [1]:
from bs4 import BeautifulSoup
from requests import get

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 
import os

In [209]:
# Function for remove comma within numbers
def removeCommas(string): 
    string = string.replace(',','')
    return string 

# Scrap data from worldmeter

In [210]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
worldometers = "https://www.worldometers.info/coronavirus/#countries"
response = get(worldometers, headers=headers)
response

<Response [200]>

In [211]:
# Scrap all content from the website
html_soup = BeautifulSoup(response.text, 'html.parser')
# After inspect the website content, data are stored inside tag 'tbody' and table header is 'thead'
table_contents = html_soup.find_all('tbody')
table_header = html_soup.find_all('thead')

# Header for the table
header = []
for head_title in table_header[0].find_all('th'):    
    header.append(str(head_title.contents))

# Save value into columns
CountryName = []
TotalCases = []
NewCases = []
TotalDeaths = []
NewDeaths = []
TotalRecovered = []
ActiveCases = []
SeriousCritical = []

for row in table_contents[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells[0].contents) > 1:
        try:
            CountryName.append(cells[0].find_all('a')[0].contents[0])
        except:
            CountryName.append(cells[0].find_all('span')[0].contents[0])
        
    else:
        CountryName.append(cells[0].contents[0])
    
    TotalCases.append(cells[1].contents[0])
    NewCases.append(cells[2].contents[0])
    TotalDeaths.append(cells[3].contents[0])
    NewDeaths.append(cells[4].contents[0])
    TotalRecovered.append(cells[5].contents[0])
    ActiveCases.append(cells[6].contents[0])    
    SeriousCritical.append(cells[7].contents[0])
        
CaseTable = pd.DataFrame({header[0]: CountryName,
                          header[1]: TotalCases,
                          header[2]: NewCases,
                          header[3]: TotalDeaths,
                          header[4]: NewDeaths,                          
                          header[5]: TotalRecovered,
                          header[6]: ActiveCases,
                          header[7]: SeriousCritical,
                          })  

CaseTable.head(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br>Deaths</br>]","['New', <br>Deaths</br>]","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
0,China,"80,754",+19,"3,136",+17,"59,912","17,706","4,794"
1,Italy,"9,172",,463,,724,"7,985",733
2,S. Korea,"7,513",+35,54,+1,247,"7,212",36
3,Iran,"7,161",,237,,"2,394","4,530",
4,France,"1,412",,30,,12,"1,370",66
5,Spain,"1,231",,30,,32,"1,169",11
6,Germany,"1,224",,2,,18,"1,204",9
7,USA,708,+4,27,+1,15,666,8
8,Diamond Princess,696,,7,,245,444,32
9,Japan,530,,9,,101,420,33


In [212]:
CaseTable.tail(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br>Deaths</br>]","['New', <br>Deaths</br>]","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
75,Tunisia,5,,,,,5,
76,Bosnia and Herzegovina,5,,,,,5,
77,French Guiana,5,,,,,5,
78,Afghanistan,4,,,,,4,
79,North Macedonia,4,,,,,4,
80,Senegal,4,,,,1,3,
81,Bulgaria,4,,,,,4,
82,Malta,4,,,,,4,
83,Bangladesh,3,,,,,3,
84,Colombia,3,,,,,3,


In [213]:
caseTableSimple = CaseTable[[CaseTable.columns[0], CaseTable.columns[1], CaseTable.columns[3], CaseTable.columns[5]]]
caseTableSimple.columns = ['Country/Region', 'Confirmed', 'Deaths', 'Recovered']
# Remove the last row of total number (changed on 20200310, worldmeter moved this row as next tbody)
#caseTableSimple = caseTableSimple.iloc[:-1,:]
# Remove lead and tail space for each element
caseTableSimple = caseTableSimple.apply(lambda x: x.str.strip())
# Remove comma for each element
caseTableSimple = caseTableSimple.applymap(removeCommas)
# Replace empty str with zero. This include row of 'Diamond Princess' (its name is empty)
caseTableSimple = caseTableSimple.replace('', '0')
# Convert data type as correct type
caseTableSimple = caseTableSimple.astype({'Country/Region':'str',
                                          'Confirmed':'int',
                                          'Deaths':'int',
                                          'Recovered':'int',                                          
                                         })
# Data for these countries come from other source
removeRegion = ['China', 'Canada', 'Australia', 'USA']
for i in removeRegion:
    caseTableSimple.drop(caseTableSimple[caseTableSimple['Country/Region'] == i].index, axis=0, inplace=True)

# Change Country name the same as my old data 
if 'S. Korea' in list(caseTableSimple['Country/Region']):
    caseTableSimple = caseTableSimple.replace('S. Korea', 'South Korea')

# Add column 'Province/State' with empty value
caseTableSimple['Province/State'] =''

# In my old data, 'Diamond Princess' is represented by 'Yokohama' in the column of 'Province/State'
if 'Diamond Princess' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Diamond Princess',].index, 'Province/State'] = 'Yokohama'
    caseTableSimple['Country/Region'].replace({'Diamond Princess':'Japan'}, inplace=True)

# In my old data, 'Belgium' has 'Brussels' in the column of 'Province/State'
if 'Belgium' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Belgium',].index, 'Province/State'] = 'Brussels'

# In my old data, I used 'Macau' not 'Macao'
if 'Macao' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Macao',].index, 'Province/State'] = 'Macau'
    caseTableSimple['Country/Region'].replace({'Macao':'Macau'}, inplace=True)

# In my old data, 'Hong Kong' has 'Hong Kong' in the column of 'Province/State'
if 'Hong Kong' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Hong Kong',].index, 'Province/State'] = 'Hong Kong'

# In my old data, 'Taiwan' has 'Taiwan' in the column of 'Province/State'
if 'Taiwan' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Taiwan',].index, 'Province/State'] = 'Taiwan'

# In my old data, I used 'United Arab Emirates' not 'UAE'
if 'UAE' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'UAE':'United Arab Emirates'}, inplace=True)

# In my old data I used US time as Last Update time
currentTime = datetime.now()
lastUpdateTime = currentTime.strftime('%m/%d/%Y %H:%M')
# Remove the first number (This only works for month number less than 10)
lastUpdateTime[1:]
caseTableSimple['Last Update'] = lastUpdateTime[1:]

# Reorder list as all old data
columnList = caseTableSimple.columns.tolist()
columnList =[columnList[i] for i in [4, 0, 5, 1, 2, 3]]
caseTableSimple = caseTableSimple[columnList]

In [214]:
caseTableSimple.tail()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
110,,Mongolia,3/10/2020 16:07,1,0,0
111,,Panama,3/10/2020 16:07,1,0,0
112,,Paraguay,3/10/2020 16:07,1,0,0
113,,St. Barth,3/10/2020 16:07,1,0,0
114,,Togo,3/10/2020 16:07,1,0,0


# Scrap data for US_CAN

In [215]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
US_Canada = "https://coronavirus.1point3acres.com/en"
response2 = get(US_Canada, headers=headers)
response2

<Response [200]>

In [216]:
# Scrap all content from the website
html_soup2 = BeautifulSoup(response2.text, 'html.parser')

In [217]:
html_soup2.find_all('span', class_='jsx-153966605')

[<span class="jsx-153966605">地区</span>,
 <span class="jsx-153966605">确诊</span>,
 <span class="jsx-153966605">新增</span>,
 <span class="jsx-153966605">死亡</span>,
 <span class="jsx-153966605">华盛顿州</span>,
 <span class="jsx-153966605">167</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">22</span>,
 <span class="jsx-153966605">加州</span>,
 <span class="jsx-153966605">143</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">2</span>,
 <span class="jsx-153966605">纽约州</span>,
 <span class="jsx-153966605">143</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">钻石公主号</span>,
 <span class="jsx-153966605">46</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">马萨诸塞</span>,
 <span class="jsx-153966605">41</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">0</span>,
 <span class="jsx-153966605">至尊公主号</sp

In [218]:
html_soup2.find_all('span', class_='jsx-3789834410')

[<span class="jsx-3789834410">地区</span>,
 <span class="jsx-3789834410">确诊</span>,
 <span class="jsx-3789834410">新增</span>,
 <span class="jsx-3789834410">死亡</span>,
 <span class="jsx-3789834410">安大略</span>,
 <span class="jsx-3789834410">35</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">不列颠哥伦比亚</span>,
 <span class="jsx-3789834410">32</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">1</span>,
 <span class="jsx-3789834410">阿尔伯塔</span>,
 <span class="jsx-3789834410">7</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">魁北克</span>,
 <span class="jsx-3789834410">4</span>,
 <span class="jsx-3789834410">0</span>,
 <span class="jsx-3789834410">0</span>]

In [219]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(0, len(html_soup2.find_all('span', class_='jsx-153966605'))-3, 4)
list2 = range(1, len(html_soup2.find_all('span', class_='jsx-153966605'))-2, 4)
list3 = range(2, len(html_soup2.find_all('span', class_='jsx-153966605'))-1, 4)
list4 = range(3, len(html_soup2.find_all('span', class_='jsx-153966605'))-0, 4)

for index in list1:
    if len(html_soup2.find_all('span', class_='jsx-153966605')[index].contents):
        Locations.append(html_soup2.find_all('span', class_='jsx-153966605')[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_='jsx-153966605')[index].contents):
        Confirmed.append(html_soup2.find_all('span', class_='jsx-153966605')[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    #if len(html_soup2.find_all('span', class_='jsx-153966605')[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_='jsx-153966605')[index].contents[0])
    #else:
    Recovered.append(0)
for index in list4:
    if len(html_soup2.find_all('span', class_='jsx-153966605')[index].contents):
        Deaths.append(html_soup2.find_all('span', class_='jsx-153966605')[index].contents[0])
    else:
        Deaths.append(0)
    
US_data = pd.DataFrame({'Province/State':Locations,
                        'Confirmed':Confirmed,
                        'Deaths':Deaths,
                        'Recovered':Recovered,  
                            })

# Remove rows that are not data
US_data.drop(US_data[US_data['Deaths'] == '死亡'].index, axis=0, inplace=True)

# Assign 0 in column Province/State as unassigned
if 0 in list(US_data['Province/State']):
    US_data.at[US_data.loc[US_data['Province/State'] == 0,].index, 'Province/State'] = 'Unassigned'

In [220]:
US_data

,Province/State,Confirmed,Deaths,Recovered
1,华盛顿州,167,22,0
2,加州,143,2,0
3,纽约州,143,0,0
4,钻石公主号,46,0,0
5,马萨诸塞,41,0,0
6,至尊公主号,21,0,0
7,乔治亚,17,0,0
8,俄勒冈,14,0,0
9,佛罗里达,14,2,0
10,德克萨斯,13,0,0


In [221]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(0, len(html_soup2.find_all('span', class_='jsx-3789834410'))-3, 4)
list2 = range(1, len(html_soup2.find_all('span', class_='jsx-3789834410'))-2, 4)
list3 = range(2, len(html_soup2.find_all('span', class_='jsx-3789834410'))-1, 4)
list4 = range(3, len(html_soup2.find_all('span', class_='jsx-3789834410'))-0, 4)

for index in list1:
    if len(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents):
        Locations.append(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents):
        Confirmed.append(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    #. They do not provide recovered cases number
    #if len(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents[0])
    #else:
    Recovered.append(0)
for index in list4:
    if len(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents):
        Deaths.append(html_soup2.find_all('span', class_='jsx-3789834410')[index].contents[0])
    else:
        Deaths.append(0)
    
CAN_data = pd.DataFrame({'Province/State':Locations,
                         'Confirmed':Confirmed,
                         'Deaths':Deaths,
                         'Recovered':Recovered,  
                            })

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Deaths'] == '死亡'].index, axis=0, inplace=True)

In [222]:
CAN_data

,Province/State,Confirmed,Deaths,Recovered
1,安大略,35,0,0
2,不列颠哥伦比亚,32,1,0
3,阿尔伯塔,7,0,0
4,魁北克,4,0,0


In [223]:
US_Can_data = pd.concat([US_data, CAN_data], ignore_index=True)

In [224]:
nameList = pd.read_csv('./web_data/statesNameTranslation.csv')

In [225]:
US_Can_data_EN = pd.merge(US_Can_data, nameList, how = 'left', left_on = 'Province/State', right_on = 'Chinese')
US_Can_data_EN = US_Can_data_EN.drop(['Chinese', 'Province/State', 'Abbr.'], axis=1)
US_Can_data_EN['Last Update'] = lastUpdateTime[1:]
US_Can_data_EN.rename(columns={'English':'Province/State'}, inplace=True)
columnOrder = ['Province/State', 'Country/Region', 'Last Update','Confirmed', 'Deaths', 'Recovered']
US_Can_data_EN = US_Can_data_EN[columnOrder]
US_Can_data_EN 

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,WA,US,3/10/2020 16:07,167,22,0
1,California,US,3/10/2020 16:07,143,2,0
2,New York,US,3/10/2020 16:07,143,0,0
3,From Diamond Princess cruise,US,3/10/2020 16:07,46,0,0
4,Massachusetts,US,3/10/2020 16:07,41,0,0
5,From Grand Princess,US,3/10/2020 16:07,21,0,0
6,Georgia,US,3/10/2020 16:07,17,0,0
7,Oregon,US,3/10/2020 16:07,14,0,0
8,Florida,US,3/10/2020 16:07,14,2,0
9,Texas,US,3/10/2020 16:07,13,0,0


In [226]:
finalTable = pd.concat([US_Can_data_EN, caseTableSimple], ignore_index=True)
finalTable

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,WA,US,3/10/2020 16:07,167,22,0
1,California,US,3/10/2020 16:07,143,2,0
2,New York,US,3/10/2020 16:07,143,0,0
3,From Diamond Princess cruise,US,3/10/2020 16:07,46,0,0
4,Massachusetts,US,3/10/2020 16:07,41,0,0
...,...,...,...,...,...,...
149,,Mongolia,3/10/2020 16:07,1,0,0
150,,Panama,3/10/2020 16:07,1,0,0
151,,Paraguay,3/10/2020 16:07,1,0,0
152,,St. Barth,3/10/2020 16:07,1,0,0


In [227]:
timeStampe = currentTime.strftime('%m_%d_%Y_%H_%M')
finalTable.to_csv('./web_data/{}_webData.csv'.format(timeStampe), index=False)

# Scrap data for China

In [74]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
CHN = "https://ncov.dxy.cn/ncovh5/view/pneumonia?scene=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0"
response3 = get(CHN, headers=headers)
response3.encoding='utf-8' ##去掉这句则乱码，加上则正常显示，其中utf-8是根据网页源代码中设置的编码格式来指定的  
response3

<Response [200]>

In [75]:
# Scrap all content from the website
html_soup3 = BeautifulSoup(response3.text, 'html.parser')

In [76]:
print(html_soup3.prettify())

<!DOCTYPE html>
<html lang="zh-cn" xmlns:layout="http://www.ultraq.net.nz/web/thymeleaf/layout">
 <head>
  <link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/umi.bundle.css?t=1583497981741" rel="stylesheet"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1,user-scalable=0,viewport-fit=cover" name="viewport"/>
  <meta content="#000000" name="theme-color"/>
  <title>
   全球新冠病毒最新实时疫情地图_丁香园
  </title>
  <script>
   window.routerBase = "/ncovh5/view";
  </script>
  <script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__ECommerce~p__Pneumonia~p__Pneumonia__area~p__Pneumonia__recommend-list~p__Pneumonia__rumo~5e297593.async.13df3f6e.js">
  </script>
  <script charset="utf-8" src="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__Pneumonia~p__Pneumonia__area~p__Pneumonia__rumor-list.async.9184546f.js">
  </script>
  <link href="//assets.dxycdn.com/gitrepo/ncov-mobile/dist/vendors~p__ECommerce~p__Pneumonia~p__Pneumonia__are

In [80]:
html_soup3.find_all('script', id='getAreaStat')[0].contents

['try { window.getAreaStat = [{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":19568,"confirmedCount":67707,"suspectedCount":0,"curedCount":45153,"deadCount":2986,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":17634,"confirmedCount":49912,"suspectedCount":0,"curedCount":29908,"deadCount":2370,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":369,"confirmedCount":3518,"suspectedCount":0,"curedCount":3024,"deadCount":125,"locationId":420900},{"cityName":"鄂州","currentConfirmedCount":352,"confirmedCount":1394,"suspectedCount":0,"curedCount":988,"deadCount":54,"locationId":420700},{"cityName":"随州","currentConfirmedCount":187,"confirmedCount":1307,"suspectedCount":0,"curedCount":1077,"deadCount":43,"locationId":421300},{"cityName":"宜昌","currentConfirmedCount":170,"confirmedCount":931,"suspectedCount":0,"curedCount":727,"deadCount":34,"l

In [ ]:
{"provinceName":"湖北省","provinceShortName":"湖北","currentConfirmedCount":19568,"confirmedCount":67707,"suspectedCount":0,"curedCount":45153,"deadCount":2986,"comment":"","locationId":420000,"statisticsData":"https://file1.dxycdn.com/2020/0223/618/3398299751673487511-135.json","cities":[{"cityName":"武汉","currentConfirmedCount":17634,"confirmedCount":49912,"suspectedCount":0,"curedCount":29908,"deadCount":2370,"locationId":420100},{"cityName":"孝感","currentConfirmedCount":369,"confirmedCount":3518,"suspectedCount":0,"curedCount":3024,"deadCount":125,"locationId":420900},{"cityName":"鄂州","currentConfirmedCount":352,"confirmedCount":1394,"suspectedCount":0,"curedCount":988,"deadCount":54,"locationId":420700},{"cityName":"随州","currentConfirmedCount":187,"confirmedCount":1307,"suspectedCount":0,"curedCount":1077,"deadCount":43,"locationId":421300},{"cityName":"宜昌","currentConfirmedCount":170,"confirmedCount":931,"suspectedCount":0,"curedCount":727,"deadCount":34,"locationId":420500},{"cityName":"荆州","currentConfirmedCount":155,"confirmedCount":1580,"suspectedCount":0,"curedCount":1376,"deadCount":49,"locationId":421000},{"cityName":"黄冈","currentConfirmedCount":151,"confirmedCount":2907,"suspectedCount":0,"curedCount":2631,"deadCount":125,"locationId":421100},{"cityName":"荆门","currentConfirmedCount":146,"confirmedCount":928,"suspectedCount":0,"curedCount":743,"deadCount":39,"locationId":420800},{"cityName":"黄石","currentConfirmedCount":95,"confirmedCount":1015,"suspectedCount":0,"curedCount":884,"deadCount":36,"locationId":420200},{"cityName":"十堰","currentConfirmedCount":93,"confirmedCount":672,"suspectedCount":0,"curedCount":571,"deadCount":8,"locationId":420300},{"cityName":"襄阳","currentConfirmedCount":82,"confirmedCount":1175,"suspectedCount":0,"curedCount":1055,"deadCount":38,"locationId":420600},{"cityName":"仙桃","currentConfirmedCount":53,"confirmedCount":575,"suspectedCount":0,"curedCount":501,"deadCount":21,"locationId":429004},{"cityName":"天门","currentConfirmedCount":24,"confirmedCount":496,"suspectedCount":0,"curedCount":457,"deadCount":15,"locationId":429006},{"cityName":"咸宁","currentConfirmedCount":21,"confirmedCount":836,"suspectedCount":0,"curedCount":801,"deadCount":14,"locationId":421200},{"cityName":"潜江","currentConfirmedCount":21,"confirmedCount":198,"suspectedCount":0,"curedCount":168,"deadCount":9,"locationId":429005},{"cityName":"恩施州","currentConfirmedCount":15,"confirmedCount":252,"suspectedCount":0,"curedCount":231,"deadCount":6,"locationId":422800},{"cityName":"神农架林区","currentConfirmedCount":0,"confirmedCount":11,"suspectedCount":0,"curedCount":11,"deadCount":0,"locationId":429021}]}